https://www.tensorflow.org/text/tutorials/classify_text_with_bert


In [ ]:
!pip install numpy==1.23

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 66.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4


In [ ]:
!pip install -q -U "tensorflow-text==2.11.*"
!pip install -q tf-models-official==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 82.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.2 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 M

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd '/content/drive/My Drive/sst2/rep/50K-2'

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
#from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

###Model setups

In [ ]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess, trainable=True)

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='sentence')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)



In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

epochs = 5
#steps_per_epoch = 1895
steps_per_epoch = 1600
batch_size = 128

#steps_per_epoch = 2
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 2.5e-6

#optimizer = optimization.create_optimizer(init_lr=init_lr,
#                                         num_train_steps=num_train_steps,
#                                          num_warmup_steps=num_warmup_steps,
#                                          optimizer_type='adagrad')

In [ ]:
classifier_model = build_classifier_model()

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
classifier_model.compile(optimizer='adam',
                         loss=loss,
                         metrics=metrics)

###Loading training data

In [ ]:
%cd '/content/drive/My Drive/sst2/rep/50K-1'


/content/drive/.shortcut-targets-by-id/1Ag1PotlFM_8hrkOc_jTYckV2clOqJ8d3/50K-1


In [ ]:
import pandas as pd
full = pd.read_csv("twitter140.csv", encoding='latin-1',dtype = str)
text_list = full.iloc[:,5]
lab_list = full.iloc[:,0]
id_list = full.iloc[:,1]
#vdata = pd.read_csv("countryInfo0-50k.csv")
#can_df = vdata[vdata["country"] == 'CA']
#ca_inds = can_df["index"].tolist()
#ca_text =[text_list[i] for i in ca_inds]
#ca_lab =[lab_list[n] for n in ca_inds]
#import re
#ca_text_clean = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",t).split()) for t in ca_text]

#au_df = vdata[vdata["country"] == 'AU']
#au_inds = au_df["index"].tolist()
#au_text =[text_list[i] for i in au_inds]
#au_text_clean = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",t).split()) for t in au_text]
#au_lab =[lab_list[n] for n in au_inds]

#nz_df = vdata[vdata["country"] == 'NZ']
#nz_inds = nz_df["index"].tolist()
#nz_text =[text_list[i] for i in nz_inds]
#nz_text_clean = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",t).split()) for t in nz_text]
#nz_lab =[lab_list[n] for n in nz_inds]

#uk_df = vdata[vdata["country"] == 'gb']
#uk_inds = uk_df["index"].tolist()
#uk_text =[text_list[i] for i in uk_inds]
#uk_lab =[lab_list[n] for n in uk_inds]

#us_df = vdata[vdata["country"] == 'US']
#us_inds = us_df["index"].tolist()
#us_text =[text_list[i] for i in us_inds]
#us_text_clean = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",t).split()) for t in us_text]
#us_lab =[lab_list[n] for n in us_inds]

In [ ]:
def pred_from_ftb(t):
  tf_batch = tokenizer(t, max_length=128, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = model(tf_batch)
  tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
  pred_labs = ['0','4']
  preds = tf.argmax(tf_predictions, axis=1)
  preds = preds.numpy()
  return [pred_labs[i] for i in preds]

def results_check(preds, labs):
    # Check if both the lists have the same length
    if len(preds) != len(labs):
        print("length different:" ,len(preds), len(labs))


    # Check if all the elements of both the lists are same
    count = 0
    results = []
    for i in range(len(preds)):
        for j in range(len(labs)):
            if preds[i] == labs[j]:
              results.append(True)
            else:
              results.append(False)
    # check if count is equal to length of list

    return results

###Random sampling from original full twitter set

In [ ]:
import random
ind_list = list(range(len(text_list)))
ind_shuffled = random.Random(333).shuffle(ind_list)

In [ ]:
ind_list[0:10]

[780161,
 1085577,
 1024094,
 553611,
 898197,
 111006,
 265269,
 1565212,
 467743,
 1364060]

In [ ]:
def convert_to_1(x):
  for o in range(len(x)):
    if x[o] == 4:
      #print("replace 4 to 1")
      x[o] = 1

##call back setting

In [ ]:
class TestCallback(tf.keras.callbacks.Callback):
    def __init__(self, path):
      #self.x = test_data[0]
      #self.y = test_data[1]
      self.path = path+'-bert-ft-twitter-50k-'
    def on_epoch_end(self, epoch, logs={}):
        #self.model.evaluate(self.x, self.y)
        #with open(self.path+"logs.txt", "w") as l:
        #  l.writelines(logs)
        print("logs: ",logs)
        saved_model_path = self.path+'-epoch' + str(epoch)
        self.model.save(saved_model_path, include_optimizer=False)
        loss_log = str(logs)
        with open(self.path+"logs.txt", "a") as l:
            l.writelines("epoch " + str(epoch) + ":" +loss_log + "\n")

In [ ]:
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('training.log')

###Model Training

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
import random
import re
#tf_cb = tf.keras.callbacks.TensorBoard(log_dir="./logs")
for a in range(4):
#for a in range(2):
  i = a+1
  print("round", i, ":")
  seeds = 111*(i+1)
  random.seed(seeds)
  ind_inds = random.sample(range(len(text_list)),160000)
  samp_inds = [ind_list[x] for x in ind_inds]
  samp_text = [text_list[i] for i in samp_inds]
  samp_text_clean = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",t).split()) for t in samp_text]
  samp_labs = [lab_list[j] for j in samp_inds]
  kf = KFold(n_splits=5)
  kf.get_n_splits(samp_inds)

  for j, (train_index, test_index) in enumerate(kf.split(samp_inds)):
    print(f"Fold {j+1}:")
    path_pref = '/content/drive/My Drive/new-train/full-r' + str(i+1)+"-fold"+ str(j+1)
    train = np.array(samp_text_clean)[train_index]
    train_lab = np.array(samp_labs, dtype = int)[train_index]
    convert_to_1(train_lab)
    #test = df_50k.iloc[test_index]
    #test.to_csv(path_pref + '-test.csv')
    #with open(path_pref+'logs.txt', "w") as l:
    #  l.writelines("logs history: \n")
    log_dir=path_pref +"-logs"
    csv_logger = tf.keras.callbacks.CSVLogger(log_dir)
    mycallback = TestCallback(path_pref)
    #fin_cb = [TestCallback, csv_logger]
    train_tf = tf.convert_to_tensor(train, dtype = None, name = "sentence")
    train_lab_tf = tf.convert_to_tensor(train_lab, dtype = None, name = "sentence")
    classifier_model = build_classifier_model()
    classifier_model.compile(optimizer='adagrad',
                         loss=loss,
                         metrics=metrics)
    tf.config.run_functions_eagerly(True)
    twitter = classifier_model.fit(x = train_tf, y = train_lab_tf, validation_split = 0.1,
                                  epochs=8, callbacks=[mycallback, csv_logger])

round 1 :
Fold 1:
Epoch 1/8


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


3600/3600 [==============================] - ETA: 0s - loss: 0.4981 - binary_accuracy: 0.7532logs:  {'loss': 0.4980945289134979, 'binary_accuracy': 0.753179669380188, 'val_loss': 0.4351877272129059, 'val_binary_accuracy': 0.7763281464576721}


3600/3600 [==============================] - 590s 163ms/step - loss: 0.4981 - binary_accuracy: 0.7532 - val_loss: 0.4352 - val_binary_accuracy: 0.7763
Epoch 2/8
3600/3600 [==============================] - ETA: 0s - loss: 0.4465 - binary_accuracy: 0.7821logs:  {'loss': 0.4464791417121887, 'binary_accuracy': 0.7821093797683716, 'val_loss': 0.41463062167167664, 'val_binary_accuracy': 0.7959374785423279}


3600/3600 [==============================] - 585s 162ms/step - loss: 0.4465 - binary_accuracy: 0.7821 - val_loss: 0.4146 - val_binary_accuracy: 0.7959
Epoch 3/8
3600/3600 [==============================] - ETA: 0s - loss: 0.4247 - binary_accuracy: 0.7950logs:  {'loss': 0.42473727464675903, 'binary_accuracy': 0.7949652671813965, 'val_loss': 0.4038223922252655, 'val_binary_accuracy': 0.8060156106948853}


...

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


3600/3600 [==============================] - ETA: 0s - loss: 0.4968 - binary_accuracy: 0.7527logs:  {'loss': 0.4968150854110718, 'binary_accuracy': 0.7526562213897705, 'val_loss': 0.448800653219223, 'val_binary_accuracy': 0.7739062309265137}


3600/3600 [==============================] - 599s 165ms/step - loss: 0.4968 - binary_accuracy: 0.7527 - val_loss: 0.4488 - val_binary_accuracy: 0.7739
Epoch 2/8
3600/3600 [==============================] - ETA: 0s - loss: 0.4430 - binary_accuracy: 0.7851logs:  {'loss': 0.4429570138454437, 'binary_accuracy': 0.7850607633590698, 'val_loss': 0.42766696214675903, 'val_binary_accuracy': 0.7916406393051147}


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-c0ad10e67b53>", line 41, in <cell line: 6>
    twitter = classifier_model.fit(x = train_tf, y = train_lab_tf, validation_split = 0.1,
  File "/usr/local/lib/python3.10/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/lib/io/file_io.py", line 221, in flush
    self._writable_file.flush()
tensorflow.python.framework.errors_impl.FailedPreconditionError: /content/drive/My Drive/new-train/full-r3-fold2-logs; Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 209

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
import random
import re
i = 2
  #j = 5
seeds = 111*(i+1)
random.seed(seeds)
ind_inds = random.sample(range(len(text_list)),160000)
samp_inds = [ind_list[x] for x in ind_inds]
samp_text = [text_list[i] for i in samp_inds]
samp_text_clean = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",t).split()) for t in samp_text]
samp_labs = [lab_list[j] for j in samp_inds]
kf = KFold(n_splits=5)
kf.get_n_splits(samp_inds)

for j, (train_index, test_index) in enumerate(kf.split(samp_inds)):
  if j >= 1:
    print(f"Fold {j+1}:")
    path_pref = '/content/drive/My Drive/new-train/full-r' + str(i+1)+"-fold"+ str(j+1)
    train = np.array(samp_text_clean)[train_index]
    train_lab = np.array(samp_labs, dtype = int)[train_index]
    convert_to_1(train_lab)
    #test = df_50k.iloc[test_index]
    #test.to_csv(path_pref + '-test.csv')
    #with open(path_pref+'logs.txt', "w") as l:
    #  l.writelines("logs history: \n")
    log_dir=path_pref +"-logs"
    csv_logger = tf.keras.callbacks.CSVLogger(log_dir)
    mycallback = TestCallback(path_pref)
    #fin_cb = [TestCallback, csv_logger]
    train_tf = tf.convert_to_tensor(train, dtype = None, name = "sentence")
    train_lab_tf = tf.convert_to_tensor(train_lab, dtype = None, name = "sentence")
    classifier_model = build_classifier_model()
    classifier_model.compile(optimizer='adagrad',
                         loss=loss,
                         metrics=metrics)
    tf.config.run_functions_eagerly(True)
    twitter = classifier_model.fit(x = train_tf, y = train_lab_tf, validation_split = 0.1,
                               epochs=8, callbacks=[mycallback, csv_logger])

Fold 2:
Epoch 1/8


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


3600/3600 [==============================] - ETA: 0s - loss: 0.4997 - binary_accuracy: 0.7867logs:  {'loss': 0.49966350197792053, 'binary_accuracy': 0.786744236946106, 'val_loss': 0.4473200738430023, 'val_binary_accuracy': 0.7825781106948853}


3600/3600 [==============================] - 578s 160ms/step - loss: 0.4997 - binary_accuracy: 0.7867 - val_loss: 0.4473 - val_binary_accuracy: 0.7826
Epoch 2/8
3600/3600 [==============================] - ETA: 0s - loss: 0.4442 - binary_accuracy: 0.7847logs:  {'loss': 0.4441637098789215, 'binary_accuracy': 0.7846527695655823, 'val_loss': 0.43673041462898254, 'val_binary_accuracy': 0.7789843678474426}


3600/3600 [==============================] - 576s 160ms/step - loss: 0.4442 - binary_accuracy: 0.7847 - val_loss: 0.4367 - val_binary_accuracy: 0.7790
Epoch 3/8
3600/3600 [==============================] - ETA: 0s - loss: 0.4235 - binary_accuracy: 0.7979logs:  {'loss': 0.4234641194343567, 'binary_accuracy': 0.7979253530502319, 'val_loss': 0.4200339615345001, 'val_binary_accuracy': 0.7994531393051147}


...

3600/3600 [==============================] - 585s 163ms/step - loss: 0.4084 - binary_accuracy: 0.8069 - val_loss: 0.4329 - val_binary_accuracy: 0.7841
Epoch 5/8
3600/3600 [==============================] - ETA: 0s - loss: 0.3961 - binary_accuracy: 0.8132logs:  {'loss': 0.39608442783355713, 'binary_accuracy': 0.8132030963897705, 'val_loss': 0.4180658757686615, 'val_binary_accuracy': 0.7915624976158142}


3600/3600 [==============================] - 586s 163ms/step - loss: 0.3961 - binary_accuracy: 0.8132 - val_loss: 0.4181 - val_binary_accuracy: 0.7916
Epoch 6/8
3600/3600 [==============================] - ETA: 0s - loss: 0.3831 - binary_accuracy: 0.8207logs:  {'loss': 0.3831004798412323, 'binary_accuracy': 0.8206857442855835, 'val_loss': 0.40723517537117004, 'val_binary_accuracy': 0.8139062523841858}


3600/3600 [==============================] - 585s 162ms/step - loss: 0.3831 - binary_accuracy: 0.8207 - val_loss: 0.4072 - val_binary_accuracy: 0.8139
Epoch 7/8
3600/3600 [==============================] - ETA: 0s - loss: 0.3724 - binary_accuracy: 0.8267logs:  {'loss': 0.37238097190856934, 'binary_accuracy': 0.826744794845581, 'val_loss': 0.4060167968273163, 'val_binary_accuracy': 0.8140624761581421}


3600/3600 [==============================] - 587s 163ms/step - loss: 0.3724 - binary_accuracy: 0.8267 - val_loss: 0.4060 - val_binary_accuracy: 0.8141
Epoch 8/8
3600/3600 [==============================] - ETA: 0s - loss: 0.3608 - binary_accuracy: 0.8331logs:  {'loss': 0.360756516456604, 'binary_accuracy': 0.8330989480018616, 'val_loss': 0.40541282296180725, 'val_binary_accuracy': 0.81640625}


3600/3600 [==============================] - 586s 163ms/step - loss: 0.3608 - binary_accuracy: 0.8331 - val_loss: 0.4054 - val_binary_accuracy: 0.8164


In [ ]:
from sklearn.model_selection import KFold
import numpy as np
import random
import re
i = 3
  #j = 5
seeds = 111*(i+1)
random.seed(seeds)
ind_inds = random.sample(range(len(text_list)),160000)
samp_inds = [ind_list[x] for x in ind_inds]
samp_text = [text_list[i] for i in samp_inds]
samp_text_clean = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",t).split()) for t in samp_text]
samp_labs = [lab_list[j] for j in samp_inds]
kf = KFold(n_splits=5)
kf.get_n_splits(samp_inds)

for j, (train_index, test_index) in enumerate(kf.split(samp_inds)):
  #if j >= 1:
    print(f"Fold {j+1}:")
    path_pref = '/content/drive/My Drive/new-train/full-r' + str(i+1)+"-fold"+ str(j+1)
    train = np.array(samp_text_clean)[train_index]
    train_lab = np.array(samp_labs, dtype = int)[train_index]
    convert_to_1(train_lab)
    #test = df_50k.iloc[test_index]
    #test.to_csv(path_pref + '-test.csv')
    #with open(path_pref+'logs.txt', "w") as l:
    #  l.writelines("logs history: \n")
    log_dir=path_pref +"-logs"
    csv_logger = tf.keras.callbacks.CSVLogger(log_dir)
    mycallback = TestCallback(path_pref)
    #fin_cb = [TestCallback, csv_logger]
    train_tf = tf.convert_to_tensor(train, dtype = None, name = "sentence")
    train_lab_tf = tf.convert_to_tensor(train_lab, dtype = None, name = "sentence")
    classifier_model = build_classifier_model()
    classifier_model.compile(optimizer='adagrad',
                         loss=loss,
                         metrics=metrics)
    tf.config.run_functions_eagerly(True)
    twitter = classifier_model.fit(x = train_tf, y = train_lab_tf, validation_split = 0.1,
                               epochs=8, callbacks=[mycallback, csv_logger])

Fold 1:
Epoch 1/8


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


3600/3600 [==============================] - ETA: 0s - loss: 0.4993 - binary_accuracy: 0.7443logs:  {'loss': 0.49928557872772217, 'binary_accuracy': 0.74434894323349, 'val_loss': 0.4463731646537781, 'val_binary_accuracy': 0.7830469012260437}


3600/3600 [==============================] - 586s 161ms/step - loss: 0.4993 - binary_accuracy: 0.7443 - val_loss: 0.4464 - val_binary_accuracy: 0.7830
Epoch 2/8
3600/3600 [==============================] - ETA: 0s - loss: 0.4447 - binary_accuracy: 0.7838logs:  {'loss': 0.4447246193885803, 'binary_accuracy': 0.7837933897972107, 'val_loss': 0.42811983823776245, 'val_binary_accuracy': 0.7915624976158142}


3600/3600 [==============================] - 585s 163ms/step - loss: 0.4447 - binary_accuracy: 0.7838 - val_loss: 0.4281 - val_binary_accuracy: 0.7916
Epoch 3/8
3600/3600 [==============================] - ETA: 0s - loss: 0.4241 - binary_accuracy: 0.7986logs:  {'loss': 0.424066424369812, 'binary_accuracy': 0.7986111044883728, 'val_loss': 0.42892348766326904, 'val_binary_accuracy': 0.7917187213897705}


...

3600/3600 [==============================] - 601s 166ms/step - loss: 0.4983 - binary_accuracy: 0.7550 - val_loss: 0.4378 - val_binary_accuracy: 0.7882
Epoch 2/8
3600/3600 [==============================] - ETA: 0s - loss: 0.4450 - binary_accuracy: 0.7855logs:  {'loss': 0.4449681043624878, 'binary_accuracy': 0.7854774594306946, 'val_loss': 0.42156273126602173, 'val_binary_accuracy': 0.794921875}


3600/3600 [==============================] - 598s 166ms/step - loss: 0.4450 - binary_accuracy: 0.7855 - val_loss: 0.4216 - val_binary_accuracy: 0.7949
Epoch 3/8
3600/3600 [==============================] - ETA: 0s - loss: 0.4239 - binary_accuracy: 0.7980logs:  {'loss': 0.42389780282974243, 'binary_accuracy': 0.7979860901832581, 'val_loss': 0.41401803493499756, 'val_binary_accuracy': 0.8010937571525574}


3600/3600 [==============================] - 599s 167ms/step - loss: 0.4239 - binary_accuracy: 0.7980 - val_loss: 0.4140 - val_binary_accuracy: 0.8011
Epoch 4/8
3600/3600 [==============================] - ETA: 0s - loss: 0.4083 - binary_accuracy: 0.8071logs:  {'loss': 0.4083487391471863, 'binary_accuracy': 0.8070659637451172, 'val_loss': 0.40722861886024475, 'val_binary_accuracy': 0.8161718845367432}


3600/3600 [==============================] - 598s 166ms/step - loss: 0.4083 - binary_accuracy: 0.8071 - val_loss: 0.4072 - val_binary_accuracy: 0.8162
Epoch 5/8
3600/3600 [==============================] - ETA: 0s - loss: 0.3970 - binary_accuracy: 0.8146logs:  {'loss': 0.39696893095970154, 'binary_accuracy': 0.8146093487739563, 'val_loss': 0.41231104731559753, 'val_binary_accuracy': 0.8010156154632568}


3600/3600 [==============================] - 600s 167ms/step - loss: 0.3970 - binary_accuracy: 0.8146 - val_loss: 0.4123 - val_binary_accuracy: 0.8010
Epoch 6/8
3289/3600 [==========================>...] - ETA: 49s - loss: 0.3843 - binary_accuracy: 0.8208

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
import random
import re
i = 4
  #j = 5
seeds = 111*(i+1)
random.seed(seeds)
ind_inds = random.sample(range(len(text_list)),160000)
samp_inds = [ind_list[x] for x in ind_inds]
samp_text = [text_list[i] for i in samp_inds]
samp_text_clean = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",t).split()) for t in samp_text]
samp_labs = [lab_list[j] for j in samp_inds]
kf = KFold(n_splits=5)
kf.get_n_splits(samp_inds)

for j, (train_index, test_index) in enumerate(kf.split(samp_inds)):
  #if j >= 1:
    print(f"Fold {j+1}:")
    path_pref = '/content/drive/My Drive/new-train/full-r' + str(i+1)+"-fold"+ str(j+1)
    train = np.array(samp_text_clean)[train_index]
    train_lab = np.array(samp_labs, dtype = int)[train_index]
    convert_to_1(train_lab)
    #test = df_50k.iloc[test_index]
    #test.to_csv(path_pref + '-test.csv')
    #with open(path_pref+'logs.txt', "w") as l:
    #  l.writelines("logs history: \n")
    log_dir=path_pref +"-logs"
    csv_logger = tf.keras.callbacks.CSVLogger(log_dir)
    mycallback = TestCallback(path_pref)
    #fin_cb = [TestCallback, csv_logger]
    train_tf = tf.convert_to_tensor(train, dtype = None, name = "sentence")
    train_lab_tf = tf.convert_to_tensor(train_lab, dtype = None, name = "sentence")
    classifier_model = build_classifier_model()
    classifier_model.compile(optimizer='adagrad',
                         loss=loss,
                         metrics=metrics)
    tf.config.run_functions_eagerly(True)
    twitter = classifier_model.fit(x = train_tf, y = train_lab_tf, validation_split = 0.1,
                               epochs=8, callbacks=[mycallback, csv_logger])